In [ ]:
# We are going to make a new name generator =) We will start with a list of names and create an autoregressive character-level language model that generates names that sound like
# the names we are given. 

#Start with importing Pytorch and matplotlib and placing the names.txt file in the same folder as this notebook
import torch
import matplotlib.pyplot as plt
words = open('names.txt', 'r').read().splitlines()

In [ ]:
words[:10]

In [ ]:
len(words)

In [ ]:
min(len(w) for w in words)

In [ ]:
max(len(w) for w in words)

In [ ]:
# This initializes a dictionary b that we can use to see some statistics about all the names. We will iterate over all two character possibilites in the names and make them a dictionary key. 
# The values increase by one when we iterate over an example of the two character pair
b = {}
for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1,ch2)
        b[bigram] = b.get(bigram,0) + 1

In [ ]:
# Here we can see the most likely two character pairs. An 'n' followed by the end of the name appears the most often. An 'a' followed by an 'n' happens the third most often.
sorted(b.items(), key = lambda kv: -kv[1])

In [ ]:
# Instead of dictionaries, Pytorch likes arrays (or tensors)
N = torch.zeros((27,27), dtype=torch.int32)

In [ ]:
# This takes all the names in the .txt file, conncatenates into one massive string, throws out duplicates (with the set function), puts them in a list, and sorts them. 
# The result is the alphabet =) We call this chars

#stoi is the mapping of each string to an integer. This assigns a number to each letter
#itos is the inverse of stoi
#Instead of having special characters for the start and end of a name, we will just use the character '.' to represent both

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
# Same method from above, but adapted for Pytorch. Remember, we want to start by understanding some statistics about the names we are given. In particular, which character (or letter) are you
# most likely to get given a letter
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]): # This is a cute way to get a list of tuples where each tuple is the letter and the letter that follows it.
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
# Lets visualize N to get a better understanding. Here we see all possible 2 character combos, with the number of times they appear. The more they appear, the more blue the box becomes.
plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i,j].item(), ha="center", va="top", color='gray')
plt.axis('off')

In [ ]:
# Ok this is great, we can now see which 2 letter pairs are most common. That means if I give you 1 letter, you can tell me which letter is most likely to follow. To make new names that
# sound real, we want to choose these most likely options. 

# Now how do we work with this distribution? We need to build a way to sample from this distribution.

#This is the first row of the matrix above. It is equivalent to N[0, :].
N[0]

In [ ]:
#This turns the first row into floats, then normalizes over the row. This is our probability distribution for the question "which character is most likely to start a name?"
p = N[0].float()
p = p / p.sum()
p

In [ ]:
# Now that we have a distribution, we can take a sample from it using Multimodial(). In this example, our character is 'J'. This represents the first character generation for a new name!
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

In [ ]:
# For a better understanding of Multimodal, see the example below. 

#g is a generator object that is useful for being deterministic, since we can use seeds. p then generates 3 random number using the g generator, and we normalize.
g = torch.Generator().manual_seed(2147483647)
p = torch.rand(3, generator=g)
p = p / p.sum()
p

In [ ]:
# Multimodal is a way to draw samples from a distribution. It will take a torch tensor probability distribution and draw a number of samples num_samples. Notice that there 
# are mostly 0s, some 1s, and few 2s. This is because the likelihood of pulling a 0 is ~60%, the liklihood of pulling a 1 is ~30%, and a 2 is ~9%. This is the distribution of tensor p above.
torch.multinomial(p, num_samples=20, replacement=True, generator=g)

In [124]:
# We have our first character. Going back to the visualized matrix above, we can take the first character and use it as a lookup. We can ask "Given this first character, what character
# is most likely to come after it?" As you can probably guess, this will become a pattern of using a generated letter to feed back into the generator to get a new letter. Let's write
# out this loop.

g = torch.Generator().manual_seed(2147483647) #Create your generator
P = (N+1).float() # N + 1 is model smoothing. The more you add, the more uniform your distribution will be. Makes it so there are no 0s in the blue matrix above.
P /= P.sum(1, keepdim=True) #Broadcasting here is incredibly tricky. Pay attention to it, and respect it. Check your work. This is also an in-place operation (/=), which doesn't create a new
# vector, and therefore saves memory.

for i in range(10):
    out = []
    ix = 0 # Start at the first index
    while True:
        p = P[ix]
        #p = torch.ones(27) / 27.0
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item() #Draw a sample from the distribution. This will serve as the next character to feed into the generator
        out.append(itos[ix])
        if ix == 0: #break case where our sampled letter is the 'end' token, meaning the end of the name
            break
    print(''.join(out))

#The name results are pretty bad lol =) But, its reasonable to assume these results are 'better' than random selection. Uncomment the #p = torch.ones(27) / 27.0 and comment the P above
# the while loop. This represents a uniform distribution, so the generator is equally likely to choose any character. Those results are much worse than the p distribution trained on 
# the bigram list of names. Bigram is still bad though =)

junide.
janasah.
p.
cony.
a.
nn.
kohin.
tolian.
juee.
ksahnaauranilevias.


In [125]:
# Great, so we now have names, but they look bad to our eye. How do we evaluate the quality of this model? We will come back to our old friend the loss function. =) This will give us a 
# single number that assesses the quality of the model. The lower the number, the better the model. The autograd repository describes the loss function in more detail, but we are essentially
# measuring the difference between the output of our model and what we want the model to ouput. If that difference is 0, then our model outputs exactly what we want =)
# A commonly used loss function is something called the 'likelihood function'. 

# Consider the following: I give you 1) a set of parameters and 2) some observed data. Assume there exists some underlying probability distribution. The Maximum Likelihood Estimation 
# is a method for estimating the parameters of the assumed probability distribution in such a way that makes the observed data the most likely. This is achieved by maximizing 
# the 'likelihood function'.

# https://towardsdatascience.com/probability-concepts-explained-maximum-likelihood-estimation-c7b4342fdbb1

# In our case:
# The model is N, the counts of each bigram.
# the model parameters are defined in the blue matrix above (the bigram probabilities)
# the probability distribution is P
# the data is words (names.txt)
# The input data is the first letter of the bigram, the desired output is the second letter of the bigram

# Using MLE, what set of parameters (bigram probabilities of blue matrix) for the distribution P is most likely resposible for producing the data in names.txt

# Andrej Karpathy explains the following:
# GOAL: maximize likelihood of the data with respect to model parameters
# equivalent to maximizing the log likelihood (because log is monotonic)
# equivalent to minimizing the negative log likelihood
# equivalent to minimizing the average negative log likelihood  
 
# Let's create this likelihood function


log_likelihood = 0.0
n = 0

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]): # This is a cute way to get a list of tuples where each tuple is the letter and the letter that follows it.
        ix1 = stoi[ch1] #input to the model
        ix2 = stoi[ch2] #desired output
        prob = P[ix1, ix2] #probability that the model assigns to this happening
        logprob = torch.log(prob) # log is a monotonic function, so we can use it to scale our function. It is more computationally efficient to do this because we can add instead of multiply
        log_likelihood += logprob
        n += 1
        #print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood # nll is negative log likelihood
print(f'{nll=}')
print(f'{nll/n}') # The lower this number, the better. nll/n is our loss function.        

log_likelihood=tensor(-559951.5625)
nll=tensor(559951.5625)
2.4543561935424805


In [ ]:
# Up until now, we have arrived at a model explicitly by doing things that "feel correct", meaning measuring and normalizing counts. Now we will take an alternative approach that
# will arrive at a similar outcome. We will cast the bigram character level language modeling into the neural network framework. The neural network will still be a bigram character level model,
# meaning it will receive a character as an input, then with the weights and biases of the network will output the probability distribution of the next character. It will makes guesses as
# to what is likely to follow the input character.

# The first thing we want to do is create the training set for the network. This code iterates over all the bigrams (x,y)
xs, ys = [], [] #xs are the inputs, ys are the targets

for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        print(ch1,ch2)
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs) # use lower case .tensor to get integers. capital T .Tensor will give float32.
ys = torch.tensor(ys)

# In this example with the name 'emma', there are 5 pieces of information. When '.' is the input, 'e' is the target. Put in terms of indexes, when 0 is the input 5 is the desired output.
# When 5 is the input 13 is the desired output, and so on.

In [ ]:
xs

In [ ]:
ys

In [ ]:
# In the context of a neural network, it doesn't make sense to feed in integers. The network is created with nodes that are multiplied by weights and scaled with biases. Think of training
# the network as tuning a vector space to be aligned with a desired outcome. We want to do this by working with vectors, not integers. Fortunatley, Pytorch has an easy way of dealing with 
# this called one hot. This takes the integer and turns the integer's index dimension into a 1. The rest are 0s. For example, an integer of 5 would result in a vector of all 0s except that
# the 5th dimension of the vector is a 1.

import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float()
xenc

In [ ]:
xenc.shape # 5x27 matrix

In [ ]:
plt.imshow(xenc) # This is the picture of the name emma =)

In [ ]:
xenc.dtype

In [ ]:
# Lets make our first neuron.
W = torch.randn((27,1)) # W is a column vector of 27 weights
xenc @ W # The @ symbol is the matrix multiplication operator in pytorch. So cool!

# Note that the resulting dot product is a 5 x 1 matrix. We took xenc which is 5 x 27 and multiplied by 27 x 1. 
# We fed all 5 inputs into a single neuron, then calculated the output with matrix multiplication of x @ W

In [ ]:
# Lets now create the 27 weights for each neuron, but also create 27 neurons. 
W = torch.randn((27,27)) 
xenc @ W

# (5, 27) @ (27, 27) -> (5, 27)

In [ ]:
# This is the firing rate of the 13th neuron, looking at the 3rd input
(xenc @ W)[3, 13]

# Same as (xenc[3] * W[:, 13]).sum()

In [ ]:
# Now we have a neural network with 27 inputs, and 27 neurons in the first layer, with only linear scaling (no bias, no squishing funciton like tanh, no other layers). Very simple.
# But our neuron values are these small float values. We want something like the original 2-dim blue matrix N. Each row told us the counts, and we can normalize each row
# to get the probabilities. However, counts aren't great to output from a neural net either. Let's instead interpret the output as log(counts).

logits = xenc @ W # log-counts
counts = logits.exp() #equivalent N
probs = counts / counts.sum(1, keepdims=True)
probs # Each row is an output from the neural net (from 1 input) and the columns are the probabilities for each next letter

# Note that each operation here is differentiable, so we can back-propogate through it

In [ ]:
probs.shape

In [ ]:
probs[0].sum()

In [ ]:
probs[0] # This is the input of a '.' into the neural net. We first got its index, then one hot encoded it, put into the neural net, and the output is the following 27 numbers. These
# numbers represent how likely each character is to come next. As we tune the weights W, we will get different probabilities out. Now the question is can we tune W so that the probabilties
# coming out are good. The way we measure good is by the loss function.

In [ ]:
# SUMMARY ------------------------------>>>>

In [ ]:
xs

In [ ]:
ys

In [ ]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g)

In [ ]:
# This represents the feed forward
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
# btw: the last 2 lines here are together called a 'softmax'

In [ ]:
probs.shape

In [ ]:
nlls = torch.zeros(5) # 5 bigrams in the name 'emma'
for i in range(5):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('--------')
  print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

In [ ]:
# --------- !!! OPTIMIZATION !!! yay --------------

# Just like in the autograd engine, we are going to optimize the neural network through gradient descent. 
# For more info, check https://github.com/ColterDowning/Neural-Networks-Zero-to-Hero/tree/master/micrograd

In [92]:
xs

tensor([ 0,  5, 13, 13,  1])

In [93]:
ys

tensor([ 5, 13, 13,  1,  0])

In [94]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [113]:
# forward pass
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
loss = -probs[torch.arange(5), ys].log().mean()

In [114]:
print(loss.item())

3.6891887187957764


In [115]:
#backward pass. First make sure to reset the gradients
W.grad = None
loss.backward()

In [116]:
W.data += -0.1 * W.grad

In [117]:
# --------- !!! OPTIMIZATION !!! yay, but this time actually --------------

In [122]:
# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

number of examples:  228146


In [123]:
# gradient descent
for k in range(100):
  
  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -50 * W.grad

# Note that the optimized loss function is about the same as the loss function from the original counting and MLE method (2.45 vs 2.49). This makes sense since we aren't taking in any
# new information. We are still just taking in a character and predicting the next one. But now, instead of just explicitly counting and normalizing, we are using gradient based learning.
# The original blue matrix is essentially the same as the fully optimized W.exp() in the gradient approach. We started with a random set and let the loss guide us to the blue matrix.

# It just so happens that the explicit approach optimizes the loss function because the setup is so simple. We can just estimate those probabilties directly and maintain in a table.
# The gradient based approach is way more flexible. We can easily scale to input multiple characters to predict the next character and complexify the net, while still outputting a single logit.
# It's not obvious how we would have extended the bigram approach to include multiple character inputs. The tables would have gotten way too large to handle because there are way too
# many possible combinations (exponential).


3.768618583679199
3.3788068294525146
3.161090850830078
3.0271859169006348
2.9344842433929443
2.867231607437134
2.8166542053222656
2.777146577835083
2.745253801345825
2.7188305854797363
2.696505308151245
2.6773719787597656
2.6608052253723145
2.6463515758514404
2.633665084838867
2.622471570968628
2.6125476360321045
2.6037068367004395
2.595794916152954
2.5886807441711426
2.5822560787200928
2.576429843902588
2.5711236000061035
2.566272735595703
2.5618226528167725
2.5577261447906494
2.5539441108703613
2.550442695617676
2.5471930503845215
2.5441699028015137
2.5413522720336914
2.538722038269043
2.536262035369873
2.5339579582214355
2.531797409057617
2.529768228530884
2.527860164642334
2.5260636806488037
2.5243704319000244
2.522773265838623
2.52126407623291
2.519836664199829
2.5184857845306396
2.5172054767608643
2.515990734100342
2.5148372650146484
2.5137407779693604
2.512697696685791
2.511704921722412
2.5107579231262207
2.509855031967163
2.5089924335479736
2.5081679821014404
2.507380485534668


In [126]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  ix = 0
  while True:
    
    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

junide.
janasah.
p.
cfay.
a.
